# dependencies
connect to cpu if want to use behaviour cloning

In [1]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [2]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 27.0 MB/s eta 0:00:00


In [3]:
!pip install imitation
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 33.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py

# environment and expert

In [4]:
import gymnasium as gym
import imitation

In [5]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
SEED = 42

env = make_vec_env(
    "seals:seals/HalfCheetah-v1", # seals:seals/CartPole-v0
    rng=np.random.default_rng(SEED),
    n_envs=8,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="seals/HalfCheetah-v1",
    venv=env,
)

In [34]:
from imitation.data import rollout

rollouts_4 = rollout.rollout(
    expert,
    env,
    rollout.make_sample_until(min_timesteps=1000, min_episodes=4), # timestep config: https://huggingface.co/HumanCompatibleAI/ppo-seals-HalfCheetah-v1
    rng=np.random.default_rng(SEED), # 随机数？ The random state to use for sampling trajectories.
)
# timestep config: https://huggingface.co/HumanCompatibleAI/ppo-seals-HalfCheetah-v1
# 4 trajectories given, each 1,000,000 timesteps

# GAIL-PPO

In [35]:
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO

learner = PPO(
    env=env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0005,
    gamma=0.95,
    n_epochs=50,
    seed=SEED,
)

reward_net = BasicRewardNet(
    observation_space=env.observation_space,
    action_space=env.action_space,
    normalize_input_layer=RunningNorm,
)

gail_trainer = GAIL(
    demonstrations=rollouts_4, # expert demo is here
    demo_batch_size=1024,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=8, #
    venv=env,
    gen_algo=learner,
    reward_net=reward_net,
    allow_variable_horizon=True,
    gen_train_timesteps = 1000
)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [29]:
expert_reward, _ = evaluate_policy(
    expert, env, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 1660.79641295 +/- 52.399360723476825


In [36]:
env.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner, env, 20, return_episode_rewards=True
)

In [37]:
gail_trainer.train(100000) #进度条显示的是gen_train_timestep除以这里的input

round:   0%|          | 0/100 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | -370     |
|    gen/time/fps             | 2695     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 6        |
|    gen/time/total_timesteps | 16384    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.608    |
|    disc/disc_acc_expert             | 0.337    |
|    disc/disc_acc_gen                | 0.88     |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.686    |
|    disc/disc_proportion_expert_pred | 0.229    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1.02e+03 |
|    disc/n_generated                 | 1.02e+03 |
-

round:   1%|          | 1/100 [01:02<1:42:56, 62.39s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | -355       |
|    gen/rollout/ep_rew_wrapped_mean | 756        |
|    gen/time/fps                    | 2484       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 6          |
|    gen/time/total_timesteps        | 32768      |
|    gen/train/approx_kl             | 0.14682981 |
|    gen/train/clip_fraction         | 0.606      |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -8.62      |
|    gen/train/explained_variance    | -0.626     |
|    gen/train/learning_rate         | 0.005      |
|    gen/train/loss                  | 0.154      |
|    gen/train/n_updates             | 50         |
|    gen/train/policy_gradient_loss  | 0.0187     |
|    gen/train/std                   | 1.01       |
|    gen/tra

round:   2%|▏         | 2/100 [02:04<1:41:48, 62.34s/it]

...

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 350        |
|    gen/rollout/ep_rew_wrapped_mean | 62.1       |
|    gen/time/fps                    | 2429       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 6          |
|    gen/time/total_timesteps        | 1572864    |
|    gen/train/approx_kl             | 0.74074465 |
|    gen/train/clip_fraction         | 0.705      |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -15        |
|    gen/train/explained_variance    | 0.495      |
|    gen/train/learning_rate         | 0.005      |
|    gen/train/loss                  | 0.249      |
|    gen/train/n_updates             | 4750       |
|    gen/train/policy_gradient_loss  | 0.0439     |
|    gen/train/std                   | 3.04       |
|    gen/tra

round:  96%|█████████▌| 96/100 [1:38:40<04:07, 61.77s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 369        |
|    gen/rollout/ep_rew_wrapped_mean | 48.6       |
|    gen/time/fps                    | 2522       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 6          |
|    gen/time/total_timesteps        | 1589248    |
|    gen/train/approx_kl             | 0.69457316 |
|    gen/train/clip_fraction         | 0.702      |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -15.1      |
|    gen/train/explained_variance    | 0.565      |
|    gen/train/learning_rate         | 0.005      |
|    gen/train/loss                  | 0.292      |
|    gen/train/n_updates             | 4800       |
|    gen/train/policy_gradient_loss  | 0.0587     |
|    gen/train/std                   | 3          |
|    gen/tra

round:  96%|█████████▌| 96/100 [1:39:02<04:07, 61.90s/it]


KeyboardInterrupt: 

In [ ]:
env.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner, env, 20, return_episode_rewards=True
)

In [ ]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

1st round:  509.06895325000005 +/- 104.65142014637341

